In [46]:
"""
Name - Matrikelnummer 
1) Pham, Ngoc Anh Trung - 7176267
2) Viktor Vironski - 4330455
3) Andy Disser - 5984875

Exercise Sheet 8
"""

from random import random
from scipy import rand
from sklearn import cluster, datasets
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl


class LinearRegression:
    """
    Linear Regression Model
    """

    def __init__(self, lr, l2_param=0, momentum=0):
        """
        Creates a linear regression model with learning rate lr

        lr : the given learning rate
        l2_param : the L2 parameter
        momentum : momentum of learning rate
        """

        self.lr = lr
        self.params_matrix = None    # matrix containing the parameters used to predict
        self.l2_param = l2_param     # L2 regularization parameter
        self.momentum = momentum

    def fit(self, X, Y):
        """
        Fit the data based on to MSE (mean squared error) using Gradient Descent

        Inputs:
        - X: the given data
        - Y: the given targets

        Output:
        - the parameter matrix (used to predict)
        """

        n = X.shape[0]      # number of trained data
        m = X.shape[1]      # dimension of trained data / number of independent variables
        p = Y.shape[1]      # dimension of targets / number of dependent variables

    
        # insert an additional column to X containing only 1
        X = np.insert(X, 0, 1, axis=1)

        epsilons = np.full(p, 0.00001)  # Criterion for stopping the loop
        i = 1

        # Need m+1 parameters for each line (each line used to predict each output variable)
        # Initialize all parameters as 1
        params_matrix = np.ones((p, m+1))
        
        # Dummy / Initialization
        prev_mse_matrix = np.full(p, np.infty)
        diff = np.full(p, np.infty)
        velocity_matrix = np.zeros((p, m+1))

        while np.all(diff >= epsilons):     # Only stopping when the difference between iterations not to large
            
            # need to build Gradient Matrix (p x m+1):
            # each row is an objective function concerning a predicted value y, 
            # each column is a differentiated variable
            temp = Y + np.multiply(np.matmul(X, np.transpose(params_matrix)), -1)
            gradient_matrix = np.divide(np.matmul(np.transpose(temp), np.multiply(X, -2)), n)

            # L2 penalty term, could be a matrix in case of more than 2 output variable
            l2_penalty = np.multiply(self.l2_param, np.apply_over_axes(np.sum, np.power(params_matrix, 2), 1))

            # Compute the mean squared error for each output value and represent it in matrix
            mse_matrix = np.divide(np.apply_over_axes(np.sum, np.power(temp, 2), 0), n) + l2_penalty

            # Difference betwwen previous and current MSE matrix
            diff = np.abs(mse_matrix-prev_mse_matrix)

            # update velocity
            velocity_matrix = np.multiply(self.lr, gradient_matrix) + np.multiply(self.momentum, velocity_matrix)

            # Update point with the learning rate, gradient descending.. (with momentum)
            params_matrix = params_matrix - velocity_matrix

            # Previous MSE matrix
            prev_mse_matrix = mse_matrix
            i += 1

        self.params_matrix = params_matrix    # Assign the parameter matrix
        
        return self.params_matrix

    def predict(self, X):
        """
        Predict

        Inputs:
        - X: the given data (test data)

        Output:
        - predicted value, store in a matrix form
        """

        # insert an additional column to X containing only 1
        X = np.insert(X, 0, 1, axis=1)

        # Predict
        predicted_matrix = np.matmul(X, np.transpose(self.params_matrix))

        return predicted_matrix


def cross_validation(train_data, train_targets, l2_param):
    """
    5-fold cross validation for L2 parameter

    Inputs:
    - train_data: the given train data
    - train_targets: the given train target
    - l2_param: the given L2 parameter

    Output:
    - score/loss value of this L2 parameter
    """

    indices = np.arange(train_data.shape[0])
    i, j =0, 0
    
    # average MSE of each iteration
    avg_mse = np.zeros(5)
    
    # Split in 5 chunks and take in each iteration a chunk as test data
    while i < 10:
        split_filter = np.logical_and(indices >= i*10, indices < (i+2)*10)
        
        cv_test_indices = indices[split_filter]
        cv_train_indices = indices[np.logical_not(split_filter)]

        cv_train = np.take_along_axis(train_data, cv_train_indices[:, np.newaxis], 0)
        cv_train_targets = np.take_along_axis(train_targets, cv_train_indices[:, np.newaxis], 0)
        
        cv_test = np.take_along_axis(train_data, cv_test_indices[:, np.newaxis], 0)
        cv_test_targets = np.take_along_axis(train_targets, cv_test_indices[:, np.newaxis], 0)

        myModel = LinearRegression(0.01, l2_param)
        myModel.fit(cv_train, cv_train_targets)
        predicted_matrix = myModel.predict(cv_test)
        
        # Take the uniform average over all mean squared errors (in case there are 
        # multiple output value) as lost score
        avg_mse[j] = np.average(np.divide(np.apply_over_axes(np.sum, \
            np.power(np.abs(cv_test_targets-predicted_matrix), 2), 0), cv_test.shape[0]).reshape(cv_test_targets.shape[1]))
        
        i += 2
        j += 1


    return np.average(avg_mse)

In [47]:
# Import and access  data
iris = datasets.load_iris()

# ==================== Exercise 1a ============================

# Create permutation of 0,1,..,149
permutation = np.random.permutation(np.arange(150))
train_filter = permutation < 100
test_filter = permutation >= 100 

# Split the data, 100 samples for training, 50 samples for testing
train_data = iris.data[train_filter][:, :2]
test_data = iris.data[test_filter][:, :2]

# The respective targets (petal length and petal width)
train_targets = iris.data[train_filter][:, 2:]
test_targets = iris.data[test_filter][:, 2:]

# Initialize the model and fit the data
myModel = LinearRegression(0.01)
myModel.fit(train_data, train_targets)

# Predict the test data
predicted_matrix = myModel.predict(test_data)

# Compute the mean squared error for test data
mse_matrix = np.divide(np.apply_over_axes(np.sum, \
    np.power(np.abs(test_targets-predicted_matrix), 2), 0), test_data.shape[0])

print("MSE of petal length and petal width respectively:\n", mse_matrix)

MSE of petal length and petal width respectively:
 [[0.56938678 0.18916271]]


In [48]:
# ==================== Exercise 1b ============================

# Initialize the model and fit the data but this time with L2 parameter of 0.3
myModel = LinearRegression(0.01, 0.3)
myModel.fit(train_data, train_targets)

# Predict the test data
predicted_matrix = myModel.predict(test_data)

# Compute the mean squared error for test data
mse_matrix = np.divide(np.apply_over_axes(np.sum, \
    np.power(np.abs(test_targets-predicted_matrix), 2), 0), test_data.shape[0])

print("MSE of petal length and petal width respectively (L2 parameter of 0.3):\n", mse_matrix)

# search in 10 different values of L2 parameter to find out which 
# perform best using cross validation for each of them
reg_params = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1])

# Vectorize the function and compute the loss of each parameter
vec_cross_validation = np.vectorize(cross_validation, signature='(m,n),(m,n),() -> ()')
results = vec_cross_validation(train_data, train_targets, reg_params)

print("The best L2 parameter sofar is ", reg_params[np.argmin(results)], " with the loss value of ", np.min(results))

MSE of petal length and petal width respectively (L2 parameter of 0.3):
 [[0.69071127 0.21007945]]
The best L2 parameter sofar is  0.5  with the loss value of  0.893194133573876


In [49]:
# ==================== Exercise 1c ============================

# Initialize the model and fit the data but this time with momentum of 0.9
myModel = LinearRegression(0.01, 0, 0.9)
myModel.fit(train_data, train_targets)

# Predict the test data
predicted_matrix = myModel.predict(test_data)

# Compute the mean squared error for test data
mse_matrix = np.divide(np.apply_over_axes(np.sum, \
    np.power(np.abs(test_targets-predicted_matrix), 2), 0), test_data.shape[0])

print("MSE of petal length and petal width respectively (momentum of 0.9):\n", mse_matrix)

MSE of petal length and petal width respectively (momentum of 0.9):
 [[0.56676815 0.18699302]]
